<h2>Insert data into an Excel file using Python</h2>

<h3>1. GENERAL INSTRUCCIONS FOR THIS PYTHON NOTEBOOK TO WORK</h3>
<h4>a. We need the following python libraries:</h4>
<ul>
  <li>pandas</li>
  <li>openpyxl</li>
  <li>mysql-connector-python</li>
</ul>

In [ ]:
# Importing our libraries
import pyodbc
import pandas as pd
import numpy as np
import openpyxl
from tabulate import tabulate

# Path
FileName = "" # Type the file name
path = rf""   # Enter the path where the file is located
tab_name_1 = f"data"  # Name of the tab with the original data
tab_name_2 = f"family" # Name of the tab with the family data

# Server settings and database
server = ''  # Server name
database = ''  # db name

# String for Windows Authentication
conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"

# Declare table names
table_data = ""
table_family = ""

    
# List of tables to process
tables = [table_data, table_family]

# Read the Excel files
df_data = pd.read_excel(path,sheet_name=tab_name_1)
df_fam = pd.read_excel(path,sheet_name=tab_name_2)


<p>CLEANING OUT THE DATA FROM THE EXCEL</p>

In [4]:
def clean_data_excel(path_to_excel, the_tab, variable_df):
    for columna in variable_df.columns:
        col_data = variable_df[columna]
        
        # Detectar columna completamente vacía (todo NaN o strings vacíos)
        if col_data.dropna().empty or (col_data.dropna() == '').all():
            variable_df[columna] = 'Null'
        elif col_data.dtype == 'object':
            variable_df[columna] = col_data.fillna('Null')
        elif np.issubdtype(col_data.dtype, np.number):
            variable_df[columna] = pd.to_numeric(col_data, errors='coerce').fillna(-9999)
        elif np.issubdtype(col_data.dtype, np.datetime64):
            variable_df[columna] = col_data.fillna(pd.Timestamp('1900-01-01'))

    with pd.ExcelWriter(path_to_excel, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
        variable_df.to_excel(writer, index=False, sheet_name=the_tab)
    
    print(f"The Excel file {path_to_excel} has been cleaned up")

# Ejecutamos la consulta
clean_data_excel(path,tab_name_1,df_data)
clean_data_excel(path,tab_name_2,df_fam)


The Excel file C:\Users\hcastillo\Downloads\Honduras\Consolidado_honduras.xlsx has been cleaned up
The Excel file C:\Users\hcastillo\Downloads\Honduras\Consolidado_honduras.xlsx has been cleaned up


<h4>4. Connecting Jupyer Notebook with MySQL Server Database</h4>

<p>The following connection uses window's authentication method which is the method utilized for connecting to the database</p>
<p>If we receive the following message: <b>You are now connected to MySQL Server Database</b>, It means the connection is successful, and we are ready to insert the data. We can use the following code below to insert the data. In this step we make sure we are authorized to connect successfully to the database.</p>

In [5]:

try:
    # test connection
    conn = pyodbc.connect(conn_str)
    print("You are now connected to MySQL Server Database")
    conn.close()
except Exception as e:
    print("Houston! there is an error!", e)

You are now connected to MySQL Server Database


<h4>5. We create the table</h4> 

In [6]:

def infer_sql_type(series, col_name):
    """Infers the SQL data type based on a pandas series."""
    if series.dropna().empty or (series.dropna() == '').all():
        return "VARCHAR(100)"  # Campo vacío o solo contiene valores vacíos
    if pd.api.types.is_integer_dtype(series):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(series):
        return "REAL"
    elif pd.api.types.is_datetime64_any_dtype(series):
        return "DATETIME" if series.dt.time.nunique() > 1 else "DATE"
    else:
        max_len = series.astype(str).str.len().max()
        return f"VARCHAR({max_len})"

def create_table_peru(df, table, conn_str):
    # Infer data types (for all except forced _id)
    column_types = {
        col: infer_sql_type(df[col], col) for col in df.columns
    }

    columns_sql = []
    primary_key_sql = ""

    for col, dtype in column_types.items():
        if col.lower() == "_id":
            columns_sql.append(f'"{col}" BIGINT')
            primary_key_sql = f'PRIMARY KEY("{col}")'
        else:
            columns_sql.append(f'"{col}" {dtype}')

    # If no _id, insert artificial ID column
    if '_id' not in df.columns and 'ID' not in df.columns:
        columns_sql.insert(0, '"ID" BIGINT IDENTITY(1,1) PRIMARY KEY NOT NULL')

    create_table_sql = f"CREATE TABLE {table} ({', '.join(columns_sql)}"
    if primary_key_sql:
        create_table_sql += f", {primary_key_sql}"
    create_table_sql += ");"

    # Connect and execute
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    conn.close()

    print(f"✅ Table '{table}' successfully created.")
    print("🔧 SQL:", create_table_sql)

# Usage
create_table_peru(df_data, table_data, conn_str)
create_table_peru(df_fam, table_family, conn_str)


✅ Table 'HND_2025_09_DTM_Data' successfully created.
🔧 SQL: CREATE TABLE HND_2025_09_DTM_Data ("start" DATETIME, "end" DATETIME, "start-geopoint" VARCHAR(49), "_start-geopoint_latitude" REAL, "_start-geopoint_longitude" REAL, "_start-geopoint_altitude" REAL, "_start-geopoint_precision" REAL, "today" DATE, "username" VARCHAR(11), "deviceid" VARCHAR(24), "International_Organization_for_Migration" VARCHAR(4), "fmp_name" VARCHAR(5), "fmp_name_001" VARCHAR(9), "fmp_name_003" VARCHAR(4), "_1_2_c_Nombre_del_punto_de_monitoreo" VARCHAR(9), "_1_2_d_Nombre_del_punto_de_monitoreo" VARCHAR(11), "_1_2_e_Nombre_del_punto_de_monitoreo" VARCHAR(10), "fmp_name_000" VARCHAR(4), "ennumerator_name_001" INTEGER, "_1_5_b_C_digo_del_encuestador" VARCHAR(4), "ennumerator_name_002_001" INTEGER, "_1_5_d_C_digo_del_encuestador" INTEGER, "_1_5_e_C_digo_del_encuestador" INTEGER, "type_movement" VARCHAR(5), "_1_8_Tipo_de_transporte" VARCHAR(8), "S001_interview_possible" VARCHAR(3), "S002_consent_form" VARCHAR(4), "

<h4>6. We cross filter the data to see what we are inserting in</h4>

In [7]:

def read_count_excel(tab):
    # Upload files to Excel with the tab name
    t = pd.read_excel(path, sheet_name=f"{tab}")

    try:
        # Mostrar los datos resultantes en formato tabular
        print(f"First five rows filtered for the tab {tab}:")
        print(tabulate(t.head(50), headers='keys', tablefmt='grid'))

        row_count = len(t)
        print(f"\nTotal number of rows that will be inserted in the DB for {tab} is: {row_count}")
        print("")

    except KeyError as e:
        print(f"Column's related error: {e}")

    except Exception as e:
        print(f"Houston, we got a problem!: {e}")

read_count_excel(tab_name_1)
read_count_excel(tab_name_2)

First five rows filtered for the tab data:
+----+----------------------------+----------------------------+-------------------------------------------------+----------------------------+-----------------------------+----------------------------+-----------------------------+---------------------+-------------+--------------------------+--------------------------------------------+------------+----------------+----------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------+------------------------+---------------------------------+----------------------------+---------------------------------+---------------------------------+-----------------+---------------------------+---------------------------+---------------------+-----------------+---------------------------+---------------------+---------------------+------------------------+-------------+--------------------------+--------+-------------

In [8]:
# Inserting data into MYSQL Server

def insert_data_from_excel(conn_str, excel_path, sheet_name, table_name):
    # Leer la hoja específica del Excel
    df = pd.read_excel(excel_path, sheet_name=sheet_name, engine='openpyxl')

    # Conectar a la base de datos
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Obtener información de las columnas de la tabla en SQL Server
    column_info_sql = f"""
        SELECT COLUMN_NAME, CHARACTER_MAXIMUM_LENGTH
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{table_name}'
    """
    cursor.execute(column_info_sql)
    column_info = {row.COLUMN_NAME: row.CHARACTER_MAXIMUM_LENGTH for row in cursor.fetchall() if row.CHARACTER_MAXIMUM_LENGTH}

    truncated_columns = []

    # Truncar valores que exceden el límite de la base de datos
    for col, max_length in column_info.items():
        if col in df.columns:
            truncated_values = df[col].astype(str).apply(lambda x: x if len(x) <= max_length else x[:max_length])
            if not df[col].equals(truncated_values):  # Detecta si hubo truncamiento
                truncated_columns.append(col)
                print(f"⚠️ Columna '{col}' truncada. Valores originales vs truncados:")
                for orig, trunc in zip(df[col], truncated_values):
                    if orig != trunc:
                        print(f"  - Original: {orig}")
                        print(f"  - Truncado: {trunc}")
                print("-" * 50)

            df[col] = truncated_values

    # Crear la sentencia SQL para inserción
    columns = ", ".join([f"[{col}]" for col in df.columns])
    placeholders = ", ".join(["?" for _ in df.columns])
    insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Insertar los datos fila por fila
    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_sql, row)

    # Confirmar cambios y cerrar conexión
    conn.commit()
    cursor.close()
    conn.close()

    print(f"✅ Datos insertados en la tabla '{table_name}' con éxito.")
    if truncated_columns:
        print(f"⚠️ Se truncaron las siguientes columnas: {', '.join(truncated_columns)}")

# Uso del código
insert_data_from_excel(conn_str, path, tab_name_1, table_data)
insert_data_from_excel(conn_str, path, tab_name_2, table_family)

✅ Datos insertados en la tabla 'HND_2025_09_DTM_Data' con éxito.
✅ Datos insertados en la tabla 'HND_2025_09_DTM_Family' con éxito.


<h3>8. Cleaning the data from the SQL Server to remove -9999, 1900-01-01, "*", and NULL text values</h3>

In [9]:
# CLEANING OUT THE TABLES 1
try:
    # Establish the connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Successfully connected to the database.")

    # Function to generate the SQL query to clean the table
    def change_to_nulls(table_data):
        
        # SQL query to clean the table
        nulls_query = f"""
            DECLARE @tableName NVARCHAR(MAX) = '{table_data}'
            DECLARE @sql NVARCHAR(MAX) = ''

            -- Generar el SQL dinámico para actualizar columnas de tipo texto
            SELECT @sql = STRING_AGG(
                'UPDATE ' + @tableName + ' SET [' + COLUMN_NAME + '] = NULL WHERE [' + COLUMN_NAME + '] = ''*'';',
                ' '
            )
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = @tableName
            AND DATA_TYPE IN ('char', 'varchar', 'nchar', 'nvarchar', 'text', 'ntext')

            -- Ejecutar el SQL dinámico
            EXEC sp_executesql @sql

            
            -- Generar el SQL dinámico para actualizar columnas de tipo fecha
            SELECT @sql = STRING_AGG(
                'UPDATE ' + @tableName + ' SET [' + COLUMN_NAME + '] = NULL WHERE [' + COLUMN_NAME + '] = ''1900-01-01'';',
                ' '
            )
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = @tableName
            AND DATA_TYPE IN ('date', 'datetime', 'datetime2', 'smalldatetime')

            -- Ejecutar el SQL dinámico
            EXEC sp_executesql @sql

        """
        return nulls_query

    # Iterate through the tables and execute the cleaning process
    for table in tables:
        query = change_to_nulls(table)
        cursor.execute(query)
        print(f"Table {table} has been cleaned up.")
        # Commit the changes
        conn.commit()
    

except Exception as e:
    # Handle errors during connection or execution
    print(f"An error occurred: {e}")

finally:
    # Ensure the connection is closed
    if 'conn' in locals():
        conn.close()
        print("Connection closed.")


Successfully connected to the database.
Table HND_2025_09_DTM_Data has been cleaned up.
Table HND_2025_09_DTM_Family has been cleaned up.
Connection closed.


In [10]:
# CLEANING OUT THE TABLES 2
try:
    # Establish the connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Successfully connected to the database.")

    # Function to generate the SQL query to clean the table
    def change_to_nulls2(table_data):
        
        # SQL query to clean the table
        nulls_query = f"""
            DECLARE @tableName NVARCHAR(MAX) = '{table_data}'
            DECLARE @sql NVARCHAR(MAX);

                        -- Ejecutar el SQL dinámico
            EXEC sp_executesql @sql

            SELECT @sql = STRING_AGG(
            CONCAT(
                'UPDATE ', @tableName, ' ',
                'SET ', QUOTENAME(c.name), ' = NULL ',
                'WHERE ', QUOTENAME(c.name), ' = ''Null'';'
            ), CHAR(13) + CHAR(10)
            )
            FROM sys.columns c
            WHERE c.object_id = OBJECT_ID(@tableName)
            AND c.system_type_id IN (167, 175, 231);
            
            EXEC sp_executesql @sql;

            """
        return nulls_query

    # Iterate through the tables and execute the cleaning process
    for table in tables:
        query = change_to_nulls2(table)
        cursor.execute(query)
        print(f"Table {table} has been cleaned up.")
        # Commit the changes
        conn.commit()
    

except Exception as e:
    # Handle errors during connection or execution
    print(f"An error occurred: {e}")

finally:
    # Ensure the connection is closed
    if 'conn' in locals():
        conn.close()
        print("Connection closed.")


Successfully connected to the database.
Table HND_2025_09_DTM_Data has been cleaned up.
Table HND_2025_09_DTM_Family has been cleaned up.
Connection closed.


In [11]:
# CLEANING OUT THE TABLES 3
try:
    # Establish the connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Successfully connected to the database.")

    # Function to generate the SQL query to clean the table
    def change_to_nulls3(table_data):
        
        # SQL query to clean the table
        nulls_query = f"""
            DECLARE @tableName NVARCHAR(MAX) = '{table_data}'
            DECLARE @sql NVARCHAR(MAX) = ''

            -- Generar el SQL dinámico para actualizar columnas numéricas
            SELECT @sql = STRING_AGG(
                'UPDATE ' + @tableName + ' SET [' + COLUMN_NAME + '] = NULL WHERE [' + COLUMN_NAME + '] = -9999;',
                ' '
            )
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = @tableName
            AND DATA_TYPE IN ('int', 'bigint', 'smallint', 'tinyint', 'decimal', 'numeric', 'float', 'real')

            -- Ejecutar el SQL dinámico
            EXEC sp_executesql @sql


            """
        return nulls_query

    # Iterate through the tables and execute the cleaning process
    for table in tables:
        query = change_to_nulls3(table)
        cursor.execute(query)
        print(f"Table {table} has been cleaned up.")
        # Commit the changes
        conn.commit()
    

except Exception as e:
    # Handle errors during connection or execution
    print(f"An error occurred: {e}")

finally:
    # Ensure the connection is closed
    if 'conn' in locals():
        conn.close()
        print("Connection closed.")


Successfully connected to the database.
Table HND_2025_09_DTM_Data has been cleaned up.
An error occurred: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot update identity column 'ID'. (8102) (SQLExecDirectW)")
Connection closed.


In [12]:
# CLEANING OUT THE TABLES 4
try:
    # Establish the connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Successfully connected to the database.")

    # Function to generate the SQL query to clean the table
    def change_to_nulls4(table_data):
        
        # SQL query to clean the table
        nulls_query = f"""

            DECLARE @tableName NVARCHAR(MAX) = '{table_data}'
            DECLARE @sql NVARCHAR(MAX) = ''

            -- Generar el SQL dinámico para actualizar columnas de tipo texto
            SELECT @sql = STRING_AGG(
                'UPDATE ' + @tableName + ' SET [' + COLUMN_NAME + '] = NULL WHERE [' + COLUMN_NAME + '] = ''-9999'';',
                ' '
            )
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = @tableName
            AND DATA_TYPE IN ('char', 'varchar', 'nchar', 'nvarchar', 'text', 'ntext')

            -- Ejecutar el SQL dinámico
            EXEC sp_executesql @sql

            """
        return nulls_query

    # Iterate through the tables and execute the cleaning process
    for table in tables:
        query = change_to_nulls4(table)
        cursor.execute(query)
        print(f"Table {table} has been cleaned up.")
        # Commit the changes
        conn.commit()
    

except Exception as e:
    # Handle errors during connection or execution
    print(f"An error occurred: {e}")

finally:
    # Ensure the connection is closed
    if 'conn' in locals():
        conn.close()
        print("Connection closed.")


Successfully connected to the database.


Table HND_2025_09_DTM_Data has been cleaned up.
Table HND_2025_09_DTM_Family has been cleaned up.
Connection closed.
